In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [3]:
import os
from langsmith_tracker import set_tracking

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name=os.getenv("LANGCHAIN_PROJECT"))

Langsmith 추적이 활성화되었습니다. [프로젝트명: LangchainNote12-RAG]


### RAG 기본 파이프라이(1~8단계)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

RAG 구조를 이해하기 위한 뼈대코드(sekeleton code) 입니다.  
각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.  
복사 해서 사용하면 됩니다.  

#### 사전단계 (pre_processing)

In [17]:
# 1단계: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf")
docs = loader.load()
# len(docs)
# docs[1].content
docs[1].metadata

{'source': 'data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf',
 'file_path': 'data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf',
 'page': 1,
 'total_pages': 32,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'user',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.14241',
 'producer': 'Hancom PDF 1.3.0.547',
 'creationDate': "D:20241106094709+09'00'",
 'modDate': "D:20241106094709+09'00'",
 'trapped': ''}

In [ ]:
# 2단계: 문서 분할(split documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
split_documents

In [16]:
# 3단계: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 4단계: DB 생성(Create DB) 및 저장, VectorStore 생성, FAISS는 메모리 공간에 저장합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

#### 런타임 단계 (runtime_processing)

In [22]:
# 5단계: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x105fe23f0>, search_kwargs={})

In [23]:
retriever.invoke("AI가 어떤 점에서 위험한가요?")

[Document(metadata={'source': 'data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf', 'file_path': 'data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf', 'page': 7, 'total_pages': 32, 'format': 'PDF 1.4', 'title': '', 'author': 'user', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.14241', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241106094709+09'00'", 'modDate': "D:20241106094709+09'00'", 'trapped': ''}, page_content='기술의 활용에 대한 위험 인식 수준은 낮은 상태로 활용 이점에 집중하는 경향\n●AI 위험 요인들에 대한 체계적 분석과 대비가 시급한 상황이나, 세계적으로 공통적으로 참고될만한 \n포괄적 분류체계가 없는 상황\n∙AI 안전과 위험 요인에 대한 정의가 부족하고, 여러 기준 및 관점에 따라 다양한 위험 요인 분류 체계가 \n혼재하고 있어 체계적인 대비가 어려움\n●따라서, 본 보고서에서는 AI 안전의 개념을 정리하고, 혼재되어 있는 위험 유형을 사례와 함께 \n분류하여 앞으로의 대응 방향과 AI 안전 확보를 위한 정책적 시사점을 제시\n6 Yoshua Bengio 외 (2024.5), International Scientific Report on the Safety of Advanced AI : INTERIM REPORT \n7 Peter Slattery 외 (2024.8), The AI Risk Repository: A Comprehensive Meta-Review, Database, and Taxonomy of \nRisks From Artificial Intelligence'),
 Document(metadata={'sou

In [26]:
# 6단계: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 7단계: LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 8단계: Chain 생성, context=chunk 묶음, quetion=user_prompt
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
question = "AI 위험 유형은?"
response = chain.stream(question)

for token in response:
    print(f"{token}", end="", flush=True)

AI 위험 유형은 다음과 같이 구분됩니다:

1. 악의적 사용 위험
2. 오작동 위험
3. 시스템적 위험
4. 교차 위험 요인

이 외에도 잘못된 정보, 악의적 행위자 및 오용, 인간-컴퓨터 상호작용, 사회경제적 및 환경적 피해, AI 시스템 안전, 실패 및 한계 등의 위험 요소가 존재합니다.

![image.png](images/langsmith_result.png)